In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
from local_utils.retrieval import get_listening_data, get_table_data

In [10]:
df_listening = get_listening_data()
artists = get_table_data(
    "artists", 
    "SELECT name, followers, popularity, genres FROM artists;"
)
df_artists = pd.DataFrame.from_records(artists)
df_artists = df_artists.rename({"name": "artist_name"}, axis=1).sort_values("artist_name")
df_artists

,artist_name,followers,popularity,genres
6,$NOT,1564795,64,None
2131,$onoma,1342,0,None
899,$uicideboy$,9089394,85,"[emo rap, horrorcore, cloud rap, trap metal, u..."
2038,03 Greedo,557544,63,None
272,100 gecs,672135,59,"[hyperpop, glitch]"
...,...,...,...,...
753,yeule,298838,56,[art pop]
1202,yksopretty,114,0,None
167,yunè pinku,33477,43,None
2210,Ásgeir,320765,52,None


In [ ]:
to_lowercase = lambda x: x.lower()
df_listening["artist_name"] = df_listening["artist_name"].apply(to_lowercase)
df_artists["artist_name"] = df_artists["artist_name"].apply(to_lowercase)
df_personal = df_listening.groupby("artist_name", as_index=False).agg(num_plays=("artist_name", "count"))
df_personal = df_personal.sort_values("num_plays", ascending=False).reset_index(drop=True)
df_personal["personal_ranking"] = df_personal.index + 1

,artist_name,num_plays,personal_ranking
0,bladee,4336,1
1,teen suicide,1602,2
2,yung lean,1512,3
3,charli xcx,1172,4
4,yeat,1082,5
...,...,...,...
4294,river whyless,1,4295
4295,kolter,1,4296
4296,koopsta knicca,1,4297
4297,boysinblush,1,4298


In [41]:
df = df_artists.merge(df_personal, how="left", on="artist_name")
df = df.dropna(subset="num_plays")
df = df.astype({"personal_ranking": int, "num_plays": int})
df

,artist_name,followers,popularity,genres,num_plays,personal_ranking
0,$not,1564795,64,None,38,513
2,$uicideboy$,9089394,85,"[emo rap, horrorcore, cloud rap, trap metal, u...",71,305
3,03 greedo,557544,63,None,3,2065
4,100 gecs,672135,59,"[hyperpop, glitch]",134,163
5,1017 alyx 9sm,15039,36,None,73,297
...,...,...,...,...,...,...
2211,xójira,32414,38,"[shoegaze, slowcore]",10,1247
2212,yellowroots,2106,20,"[norsk rock, norwegian pop]",4,1848
2213,yeule,298838,56,[art pop],21,789
2215,yunè pinku,33477,43,None,20,854
